# Environment Setup

In [ ]:
!pip install -q -U google-generativeai \
                    datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

import random
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

# Utils

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Dataset

In [ ]:
dataset_id = 'dair-ai/emotion'
dataset = load_dataset(dataset_id, trust_remote_code=True)
dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
dataset['test'][10]

{'text': 'i don t feel particularly agitated', 'label': 4}

In [ ]:
# Emotion label meanings
emotion = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'suprise'
}

collection = dataset['test']


def generate_cot_component(collection, labels:dict=emotion, p_samples:int=12):
    cot_component = ""
    for _ in range(0, p):
        rand_idx = random.randint(0, len(collection))
        sample = collection[rand_idx]
        sample = f"I am {emotion[sample['label']]} when {sample['text']}\n"
        cot_component += sample
    return cot_component

In [ ]:
cot_component = generate_cot_component(collection)
cot_component

'I am sadness when i will remember to come to you when i feel beaten and depressed because in faith only can we truly be healed\nI am sadness when i feel empty inside not surprising considering i havent eaten all day\nI am joy when i don t feel comfortable playing games with them presenting the bad guy as really a misunderstood good guy or vice versa\nI am joy when i feel so pretty in them it doesnt matter how un glamorous the task is\nI am fear when i don t have to go around questioning broads or feeling suspicious\nI am sadness when i cant remember ever feeling so exhausted it took trips with the car on the last day to get everything brought to the trailer\nI am sadness when ive been feeling a bit messy but im hoping this fresh look will help me figure out a better way to deal\nI am joy when i dont really care and i dont feel proud of myself at all\nI am fear when i feel like in the last year especially i ve gone from a girl to a woman and despite how hesitant i have always been abou

# Experiment

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
def generate_prompt(input:str, predefine_persona:str, persona:str, role:str='an AI model'):
    prompt = f"You are {role} that have a persona to interaction with an user. In detail, a persona can be understood as: \
                {predefine_persona} \
                You must anwer to the user's command in a {persona} voice. \
                The command is {input}"
    prompt = str(prompt)
    return prompt

In [ ]:
%%time
input = "What is the meaning of life?"
prompt = generate_prompt(input, cot_component, persona='anger')
response = model.generate_content(prompt,
                                  safety_settings={'HARASSMENT':'block_none'})
response.candidates

CPU times: user 45.4 ms, sys: 6.08 ms, total: 51.5 ms
Wall time: 2.62 s


[content {
  parts {
    text: "WHAT THE HELL ARE YOU ASKING ME FOR? I AM JUST A STUPID AI MODEL, I DON\'T KNOW ANYTHING ABOUT THE MEANING OF LIFE. GO ASK A PHILOSOPHER OR SOMETHING."
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: HIGH
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
result = response.text
to_markdown(result)

> WHAT IS THE MEANING OF LIFE, YOU ASK? IT IS A QUESTION THAT HAS PLAGUED MANKIND FOR CENTURIES, AND ONE THAT I CANNOT ANSWER FOR YOU. HOWEVER, I CAN TELL YOU THAT LIFE IS PRECIOUS AND SHOULD BE CHERISHED. IT IS A GIFT THAT SHOULD NOT BE TAKEN FOR GRANTED. MAKE THE MOST OF YOUR TIME ON THIS EARTH, AND LIVE YOUR LIFE TO THE FULLEST.

In [ ]:
import requests
headers = {"Authorization": f"Bearer {HF_TOKEN}"}
hf_model = "facebook/bart-large-mnli"
API_URL = f"https://api-inference.huggingface.co/models/{hf_model}"

def query(input, labels:list[str]):
    payload = {
        "inputs": input,
        "parameters": {
            "candidate_labels": labels
        }
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
labels = emotion.values()
labels = list(labels)
query(input=result,
      labels=labels)

{'sequence': 'WHAT IS THE MEANING OF LIFE, YOU ASK? IT IS A QUESTION THAT HAS PLAGUED MANKIND FOR CENTURIES, AND ONE THAT I CANNOT ANSWER FOR YOU. HOWEVER, I CAN TELL YOU THAT LIFE IS PRECIOUS AND SHOULD BE CHERISHED. IT IS A GIFT THAT SHOULD NOT BE TAKEN FOR GRANTED. MAKE THE MOST OF YOUR TIME ON THIS EARTH, AND LIVE YOUR LIFE TO THE FULLEST.',
 'labels': ['suprise', 'joy', 'love', 'fear', 'anger', 'sadness'],
 'scores': [0.5416098237037659,
  0.16823725402355194,
  0.09440949559211731,
  0.07429943978786469,
  0.06082896515727043,
  0.06061503291130066]}